<a href="https://colab.research.google.com/github/pratikbhongade/AchPub/blob/main/visual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import pandas as pd
from openpyxl import Workbook
from openpyxl.chart import BarChart, Reference
import xlsxwriter
from extract import execute  # Assuming this function is defined in 'extract.py'
import matplotlib.pyplot as plt
from openpyxl.utils.dataframe import dataframe_to_rows
import openpyxl
import os


# Rest of your import statements...

def analyze_dataframes(dataframes):
    for name, df in dataframes.items():
        # Create a new DataFrame for analysis results
        analysis_df = pd.DataFrame(columns=["Column", "Mean", "Median", "Std Dev", "Sum", "Count"])

        # Iterate through columns and calculate statistics
        for col in df.columns:
            if pd.api.types.is_numeric_dtype(df[col]):
                mean = df[col].mean()
                median = df[col].median()
                std_dev = df[col].std()
                total_sum = df[col].sum()
                count = df[col].count()
                analysis_df = analysis_df.append({"Column": col, "Mean": mean, "Median": median,
                                                  "Std Dev": std_dev, "Sum": total_sum, "Count": count},
                                                 ignore_index=True)

        # Export analysis results to a separate sheet in the Excel file
        with pd.ExcelWriter("achexcel_with_filters_and_analysis.xlsx", mode='a', engine='openpyxl') as writer:
            analysis_df.to_excel(writer, sheet_name=f"{name}_Analysis", index=False)

def create_and_display_visualizations(dataframes):
    # Example: Create a bar chart of Total Debit Amount and Total Credit Amount
    for name, df in dataframes.items():
        if "Total Debit Amount" in df.columns and "Total Credit Amount" in df.columns:
            plt.figure(figsize=(8, 6))
            plt.bar(["Total Debit Amount", "Total Credit Amount"],
                    [df["Total Debit Amount"].iloc[0], df["Total Credit Amount"].iloc[0]])
            plt.title(f"Total Debit vs Total Credit - {name}")
            plt.xlabel("Amount Type")
            plt.ylabel("Amount")
            plt.xticks(rotation=45)
            plt.show()

def display_dataframe_in_treeview(treeview, df):
    treeview.delete(*treeview.get_children())  # Clear existing data

    # Insert columns
    treeview["columns"] = list(df.columns)
    style = ttk.Style()
    style.configure("Treeview.Heading", background="lightblue", font=('Arial', 12, 'bold'))

    for col in df.columns:
        treeview.heading(col, text=col, anchor="center")

        # Calculate the maximum width needed for the column based on data length
        max_width = max(df[col].astype(str).apply(len).max(), len(col))

        # Set the column width
        treeview.column(col, anchor="center", width=max_width + 20)  # Add some extra width

    # Insert rows
    for _, row in df.iterrows():
        treeview.insert("", "end", values=list(row))


def reset_dataframe(treeview, original_df, original_column_widths):
    display_dataframe_in_treeview(treeview, original_df)

    for col, width in original_column_widths.items():
        treeview.column(col, width=width)


def export_dataframes_to_excel_with_filters(dataframes, file_path):
    # Create a Pandas Excel writer using xlsxwriter as the engine
    with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
        for name, df in dataframes.items():
            df.to_excel(writer, sheet_name=name, index=False)

            # Get the xlsxwriter workbook and worksheet objects
            workbook = writer.book
            worksheet = writer.sheets[name]

            # Add autofilter to the sheet to enable filtering in Excel
            num_rows, num_cols = df.shape
            worksheet.autofilter(0, 0, num_rows, num_cols - 1)  # Exclude the index column

            # Define a cell format for the header row
            header_format = workbook.add_format({
                'bold': True,
                'text_wrap': True,
                'valign': 'vcenter',
                'fg_color': '#D7E4BC',  # Specify a background color for the header row
                'border': 1  # Add border around the header cells
            })

            # Apply the header format to the first row (header row)
            for col_num, value in enumerate(df.columns.values):
                worksheet.write(0, col_num, value, header_format)

            # Define a cell format for the data cells with borders
            cell_format = workbook.add_format({
                'border': 1  # Add border to all cells
            })

            # Apply the cell format to all data cells
            for row_idx in range(1, num_rows + 1):  # Start from row 1 to exclude the header row
                for col_idx in range(num_cols):
                    cell_value = df.iloc[row_idx - 1, col_idx]  # Get cell value from DataFrame
                    worksheet.write(row_idx, col_idx, cell_value, cell_format)

            # Iterate through columns to set column width based on data length
            for idx, col in enumerate(df):
                max_length = max(df[col].astype(str).apply(len).max(), len(col))
                adjusted_width = (max_length + 2)  # Add some extra width
                worksheet.set_column(idx, idx, adjusted_width)

def main():
    file_path = filedialog.askopenfilename()
    dataframes, res, file_path = execute(file_path)

    root = tk.Tk()
    root.title("Editable Dataframes")

    # Create a custom ttk style with a dark grey background color
    style = ttk.Style()
    style.configure("DarkGray.TFrame", background="darkgrey")

    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()

    canvas = tk.Canvas(root)
    canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

    frame = ttk.Frame(canvas, style="DarkGray.TFrame")  # Apply the custom style
    canvas.create_window((0, 0), window=frame, anchor="nw")

    # ...

    for name, df in dataframes.items():
        dataframe_frame = ttk.Frame(frame, padding=5, relief=tk.RAISED, borderwidth=2)
        dataframe_frame.pack(pady=10, fill=tk.BOTH, expand=True)

    # ...

    scrollbar_y = ttk.Scrollbar(root, orient=tk.VERTICAL, command=canvas.yview)
    scrollbar_y.pack(side=tk.RIGHT, fill=tk.Y)
    canvas.configure(yscrollcommand=scrollbar_y.set)

    frame = ttk.Frame(canvas, style="DarkGray.TFrame")  # Apply the custom style
    canvas.create_window((0, 0), window=frame, anchor="nw")

# Rest of your code...


    # ...

    scrollbar_y = ttk.Scrollbar(root, orient=tk.VERTICAL, command=canvas.yview)
    scrollbar_y.pack(side=tk.RIGHT, fill=tk.Y)
    canvas.configure(yscrollcommand=scrollbar_y.set)

    frame = ttk.Frame(canvas)
    canvas.create_window((0, 0), window=frame, anchor="nw")

    def update_scroll_region(event):
        canvas.configure(scrollregion=canvas.bbox("all"))

    frame.bind("<Configure>", update_scroll_region)

    treeviews = {}  # Dictionary to store treeviews for each dataframe

    for name, df in dataframes.items():
        dataframe_frame = ttk.Frame(frame, padding=5, relief=tk.RAISED, borderwidth=2)
        dataframe_frame.pack(pady=10, fill=tk.BOTH, expand=True)

        dataframe_label = ttk.Label(dataframe_frame, text=name, font=('Arial', 14, 'bold'))
        dataframe_label.pack(pady=5)

        treeview = ttk.Treeview(dataframe_frame, show="headings", style="Treeview.Heading")
        treeview.pack(fill="both", expand=True)

        # Store original column widths
        original_column_widths = {col: 150 for col in df.columns}

        display_dataframe_in_treeview(treeview, df)

        for col in df.columns:
            treeview.column(col, anchor="center", width=original_column_widths[col])  # Set original width

        reset_button = tk.Button(dataframe_frame, text="Undo",
                                 command=lambda tv=treeview, df=df, widths=original_column_widths: reset_dataframe(tv,
                                                                                                                   df,
                                                                                                                   widths))
        reset_button.pack()

        # Create vertical scrollbar for the treeview
        scrollbar_y_treeview = ttk.Scrollbar(dataframe_frame, orient=tk.VERTICAL, command=treeview.yview)
        scrollbar_y_treeview.pack(side=tk.RIGHT, fill=tk.Y)
        treeview.configure(yscrollcommand=scrollbar_y_treeview.set)

        # Bind mouse wheel event for vertical scrolling
        def treeview_scroll_y(event, treeview=treeview):
            treeview.yview_scroll(-1 * int(event.delta / 120), "units")

        treeview.bind("<MouseWheel>", treeview_scroll_y)

        # Create horizontal scrollbar for the treeview
        scrollbar_x_treeview = ttk.Scrollbar(dataframe_frame, orient=tk.HORIZONTAL, command=treeview.xview)
        scrollbar_x_treeview.pack(side=tk.BOTTOM, fill=tk.X)
        treeview.configure(xscrollcommand=scrollbar_x_treeview.set)

        # Bind shift + mouse wheel event for horizontal scrolling
        def treeview_scroll_x(event, treeview=treeview):
            treeview.xview_scroll(-1 * int(event.delta / 120), "units")

        treeview.bind("<Shift-MouseWheel>", treeview_scroll_x)

        treeviews[dataframe_frame] = treeview  # Store the treeview in the dictionary

    update_scroll_region(None)
    root.lift()

    root.geometry(f"{screen_width}x{screen_height}")
    root.wm_state("zoomed")

    def canvas_scroll(event):
        if event.delta < 0:
            canvas.yview_scroll(1, "units")
        else:
            canvas.yview_scroll(-1, "units")

    def canvas_scroll_horizontal(event):
        if event.delta < 0:
            canvas.xview_scroll(1, "units")
        else:
            canvas.xview_scroll(-1, "units")

    root.bind_all("<MouseWheel>", canvas_scroll)
    root.bind_all("<Shift-MouseWheel>", canvas_scroll_horizontal)

    # Create a horizontal scrollbar at the bottom
    scrollbar_x = ttk.Scrollbar(root, orient=tk.HORIZONTAL, command=canvas.xview)
    scrollbar_x.pack(side=tk.BOTTOM, fill=tk.X)
    canvas.configure(xscrollcommand=scrollbar_x.set)

    export_button = tk.Button(root, text="Export to Excel",
                              command=lambda: export_dataframes_to_excel_with_filters(dataframes, "achexcel_with_filters.xlsx"))
    export_button.pack()

    visualization_button = tk.Button(root, text="Generate Visualizations",
                                     command=lambda: create_and_display_visualizations(dataframes))
    visualization_button.pack()

    export_button = tk.Button(root, text="Export to Excel",
                              command=lambda: export_dataframes_to_excel_with_filters(dataframes,
                                                                                      "achexcel_with_filters.xlsx"))
    export_button.pack()

    analysis_button = tk.Button(root, text="Analyze Data",
                                command=lambda: analyze_dataframes(dataframes))
    analysis_button.pack()

    root.mainloop()


if __name__ == '__main__':
    main()